In [2]:
!pip install accelerate peft bitsandbytes transformers trl

In [3]:
!pip install --upgrade datasets accelerate bitsandbytes

In [4]:
# Model from Hugging Face hub
base_model = "NousResearch/Llama-2-7b-chat-hf"

# Fine-tuned model
new_model = "./results/llama-base-simple-format"


In [5]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

In [6]:
#!pip install bitsandbytes==0.39.0

In [7]:
''' for GPU
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)'''

'\ncompute_dtype = getattr(torch, "float16")\n\nquant_config = BitsAndBytesConfig(\n    load_in_4bit=True,\n    bnb_4bit_quant_type="nf4",\n    bnb_4bit_compute_dtype=compute_dtype,\n    bnb_4bit_use_double_quant=False,\n)'

In [8]:
#!pip install accelerate==0.21.0

In [9]:
#!pip install -U accelerate

In [10]:
#!pip install -i https://pypi.org/simple/ bitsandbytes

In [11]:
base_data = "./data/base0518_inout_train.csv"

In [12]:
import pandas as pd


df = pd.read_csv(base_data)


print(df.head())



                                               input  \
0  [INST]Given a raw source for a product, genera...   
1  [INST]Given a raw source for a product, genera...   
2  [INST]Given a raw source for a product, genera...   
3  [INST]Given a raw source for a product, genera...   
4  [INST]Given a raw source for a product, genera...   

                                              output  
0                      여성용 플리스 미니드레스 가을겨울 슬림핏 이너 스커트  
1                       빈티지 스타일 남성 가죽 자켓 가을 겨울 패션 외투  
2  [스마트 청소 로봇, 로봇 물걸레 청소기, 전자동 진공 청소기, 가정용 로봇 청소기...  
3  [여행용배낭, 대용량배낭, 체크무늬배낭, 방수배낭, 남녀공용배낭, 2023신모델배낭...  
4                   2023 신상 남성 청바지 와이드핏 스트레이트 빈티지 워싱  


In [13]:
data = load_dataset("csv", data_files="./data/base0518_inout_train.csv")
val_data =load_dataset("csv", data_files="./data/base0518_inout_val.csv")
data

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 3977
    })
})

In [14]:
data['train'][0]

{'input': '[INST]Given a raw source for a product, generate a coherent and relevant product title in Korean.[/INST]플리스 미니드레스 여성용 이너 슬림핏 블랙 미니스커트 가을 겨울 신상 슬림핏 이너 세트',
 'output': '여성용 플리스 미니드레스 가을겨울 슬림핏 이너 스커트'}

In [15]:
data['train'][1]

{'input': '[INST]Given a raw source for a product, generate a coherent and relevant product title in Korean.[/INST]유럽과 미국 스타일 빈티지 가죽 자켓 남성 가을 겨울 신상품 스탠드 칼라 오토바이 외투 젊은이 트렌디 패션 가죽',
 'output': '빈티지 스타일 남성 가죽 자켓 가을 겨울 패션 외투'}

In [16]:
device = "cuda" if torch.cuda.is_available() else "cpu" #will not use gpu right now
device ="cpu"
device

'cpu'

In [17]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    #quantization_config=quant_config,
    #device_map = "auto"
    #device_map={"": 0}
    device_map="cpu"
)
#device = torch.device('cpu')
#model = model.to(device)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/Users/kangwoojin/.pyenv/versions/3.11.5/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/Users/kangwoojin/.pyenv/versions/3.11.5/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/Users/kangwooji

In [18]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [19]:
def tokenize_function(examples):
    inputs = tokenizer(examples["input"], padding="max_length", truncation=True, max_length=512)
    outputs = tokenizer(examples["output"], padding="max_length", truncation=True, max_length=512)
    inputs['labels'] = outputs['input_ids']
    return inputs

In [20]:
data['train'].column_names

['input', 'output']

In [21]:
data['train'][0]

{'input': '[INST]Given a raw source for a product, generate a coherent and relevant product title in Korean.[/INST]플리스 미니드레스 여성용 이너 슬림핏 블랙 미니스커트 가을 겨울 신상 슬림핏 이너 세트',
 'output': '여성용 플리스 미니드레스 가을겨울 슬림핏 이너 스커트'}

In [22]:
tokenized_train_dataset = data['train'].map(tokenize_function, batched=True)
tokenized_val_dataset = val_data['train'].map(tokenize_function, batched=True)

Map:   0%|          | 0/3977 [00:00<?, ? examples/s]

Map:   0%|          | 0/442 [00:00<?, ? examples/s]

In [23]:
tokenized_train_dataset

Dataset({
    features: ['input', 'output', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 3977
})

In [24]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [25]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=7,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard",
    no_cuda=True #train in CPU
)

training_params = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=7,
    weight_decay=0.01,
    #remove_unused_columns=False
)

/Users/kangwoojin/.pyenv/versions/3.11.5/lib/python3.11/site-packages/transformers/training_args.py:1489: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
/Users/kangwoojin/.pyenv/versions/3.11.5/lib/python3.11/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [26]:
from transformers import Trainer


In [27]:
'''
trainer = SFTTrainer(
    model=model,
    train_dataset=data['train'],
    eval_dataset=val_data['train'],
    #peft_config=peft_params,
    dataset_text_field="preprocessed_input",
    #max_seq_length=512,
    tokenizer=tokenizer,
    args=training_params,
    #packing=False,
)
'''
trainer = Trainer(
    model=model,
    train_dataset=data['train'],
    eval_dataset=val_data['train'],
    #peft_config=peft_params,
    #dataset_text_field="preprocessed_input",
    #max_seq_length=512,
    tokenizer=tokenizer,
    args=training_params,
    #packing=False,
)

In [28]:
print(data['train'][:5])
print(val_data['train'][:5])


{'input': ['[INST]Given a raw source for a product, generate a coherent and relevant product title in Korean.[/INST]플리스 미니드레스 여성용 이너 슬림핏 블랙 미니스커트 가을 겨울 신상 슬림핏 이너 세트', '[INST]Given a raw source for a product, generate a coherent and relevant product title in Korean.[/INST]유럽과 미국 스타일 빈티지 가죽 자켓 남성 가을 겨울 신상품 스탠드 칼라 오토바이 외투 젊은이 트렌디 패션 가죽', '[INST]Given a raw source for a product, generate related search keywords in Korean that are relevant to the raw source and practically usable[/INST]다기능 스마트 청소 로봇 가정용 전자동 물걸레 청소기 진공 청소기 조용하고 얇은 바닥 청소 장비', '[INST]Given a raw source for a product, generate related search keywords in Korean that are relevant to the raw source and practically usable[/INST]레트로 체크무늬 대용량 여행 배낭 남녀공용 2023 신모델 방수 내마모성 다기능 학생 책가방', '[INST]Given a raw source for a product, generate a coherent and relevant product title in Korean.[/INST]청바지 남성 2023 봄 신상 트렌디한 와이드핏 스트레이트 빈티지 워싱 주름 방지 내마모성 남성용 캐주얼 롱팬츠'], 'output': ['여성용 플리스 미니드레스 가을겨울 슬림핏 이너 스커트', '빈티지 스타일 남성 가죽 자켓 가을 겨울 패션 외투', '[스마트 청소

In [29]:
trainer.train()

RuntimeError: MPS backend out of memory (MPS allocated: 17.75 GB, other allocations: 324.07 MB, max allowed: 18.13 GB). Tried to allocate 172.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:
new_model = "./results/llama-0519"

In [ ]:
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)


In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt1 = "[INST]Given a raw source for a product, generate a coherent and relevant product title in Korean.[/INST]새로운 스마트워치 2023 터치 스크린 운동 모니터링 심박수 혈압 수면 추적 다기능 남녀 공용 블루투스 팔찌"


In [ ]:
prompt2 = "[INST]Given a raw source for a product, generate five detailed descriptions in Korean. Each description should be korean and relevant to the raw source.[/INST]창의적인 다기능 책상 정리함 사무실 학생 문구함 가정용 미니 서랍형 화장품 정리함"


In [ ]:
#prompt = "[INST]Given a raw source for a product, generate five detailed descriptions in Korean.[/INST]창의적인 다기능 책상 정리함 사무실 학생 문구함 가정용 미니 서랍형 화장품 정리함"

input_text = f"{prompt1}"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

with torch.no_grad():
    output_ids = model.generate(
        input_ids,
        max_length=512,
        do_sample=True,  # Enable sampling if you want diverse results
        temperature=0.9,  # Adjust based on desired creativity
        top_p=0.95,
        top_k=50
    )
response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(response)

In [ ]:
input_text = f"{prompt2}"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

with torch.no_grad():
    output_ids = model.generate(
        input_ids,
        max_length=512,
        do_sample=True,  # Enable sampling if you want diverse results
        temperature=0.9,  # Adjust based on desired creativity
        top_p=0.95,
        top_k=50
    )
response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(response)

[{'generated_text': '<s>[INST]Given a raw source for a product, generate a coherent and relevant product title in Korean. Just give me answer[/INST][INPUT]새로운 스마트워치 2023 터치 스크린 운동 모니터링 심박수 혈압 수면 추적 다기능 남녀 공용 블루투스 팔찌[/INPUT]  Sure, here\'s a possible Korean product title for the raw source you provided:\n\n"2023 터치 스크린 운동 모니터링 심박수 혈압 수면 추적 다기능 남녀 공용 블루투스 팔찌"\n\nTranslation: "2023 Touch Screen Exercise Monitoring Heart Rate Blood Pressure Sleep Tracker Multi-Functional Unisex BlueTooth Wristband"\n\nThis title is coherent and relevant because it accurately reflects the product\'s features and functionalities, while also incorporating relevant keywords that users might search for when looking for a product like this in Korean.'}]

저장한 모델 불러와보기

In [ ]:
#from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
#my_model_path = '/content/drive/MyDrive/Colab Notebooks/창통설_그로들/results/llama-7b-chat-productname-test'
#my_model = AutoModelForCausalLM.from_pretrained(my_model_path)
#my_tokenizer = AutoTokenizer.from_pretrained(my_model_path)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
'''
logging.set_verbosity(logging.CRITICAL)

prompt = "Generate product name in Korean from this raw source: 보스 헤드폰 회색 사운드 음질 폭신"
pipe = pipeline(task="text-generation", model=my_model, tokenizer=my_tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])
'''